<a href="https://colab.research.google.com/github/JorgeZarama/TFM-final-models/blob/main/Sentiment_140_Regresi%C3%B3n_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análisis de Sentimiento Conjunto de Datos Sentiment 140

Vamos a ocupar el Sentiment 140
https://www.kaggle.com/datasets/kazanova/sentiment140/code

In [ ]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os
import numpy as np
import pandas as pd
import requests
import http.client
import io
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
import re
from sklearn.linear_model import LogisticRegression
import joblib
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import time

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
from wordcloud import WordCloud

import spacy
import nltk
nltk.download('wordnet')
#nlp=spacy.load('en_core_web_sm')
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Embedding,SimpleRNN,LSTM,GRU,Bidirectional,BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from gensim.models import Word2Vec

import nltk
import subprocess

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jorge\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
ruta = "training.1600000.processed.noemoticon.csv"
encoding = "ISO-8859-1"
df = pd.read_csv(ruta, encoding=encoding, header=None, names=['target','ids','date','flag','user','text'])
display(df.sample(10))

,target,ids,date,flag,user,text
1476890,4,2066175568,Sun Jun 07 10:01:44 PDT 2009,NO_QUERY,pisces_mami,@jaymoney4 awwww... Poor babez!! Lol. Nah it i...
521393,0,2192400689,Tue Jun 16 06:59:02 PDT 2009,NO_QUERY,remedii28,sooo tiireedd qonna meet up wit one of the be...
1440418,4,2061666031,Sat Jun 06 21:20:58 PDT 2009,NO_QUERY,guruvan,@Urvaksh There will be several of us there. I ...
1329689,4,2015697259,Wed Jun 03 05:18:35 PDT 2009,NO_QUERY,BrowncoatBuffy,@dollylolly I shall make my presence made soon...
159313,0,1956810573,Thu May 28 22:44:56 PDT 2009,NO_QUERY,jennaeccles,@hughlaurie I am so mad that House is over c...
8879,0,1548521469,Fri Apr 17 21:11:07 PDT 2009,NO_QUERY,FerdiieJonas,Homework
80947,0,1752529045,Sat May 09 22:10:42 PDT 2009,NO_QUERY,joliesgrrl80,got sunburned from the dodger game. boo.
1472565,4,2065397606,Sun Jun 07 08:30:43 PDT 2009,NO_QUERY,carlinhaanyyy,@taylorswift13 http://twitpic.com/6g8k2 - You ...
1376619,4,2051774560,Fri Jun 05 22:46:43 PDT 2009,NO_QUERY,preethzzz,@rampantheart good day to you!
1527984,4,2177295417,Mon Jun 15 05:50:44 PDT 2009,NO_QUERY,Kelliekk,@LoveMeSomeDean http://twitpic.com/7g0h9 - oh ...


In [ ]:
#Sacamos Información
display(df.describe(include='all'))


,target,ids,date,flag,user,text
count,1.600000e+06,1.600000e+06,1600000,1600000,1600000,1600000
unique,NaN,NaN,774363,1,659775,1581466
top,NaN,NaN,Mon Jun 15 12:53:14 PDT 2009,NO_QUERY,lost_dog,isPlayer Has Died! Sorry
freq,NaN,NaN,20,1600000,549,210
mean,2.000000e+00,1.998818e+09,NaN,NaN,NaN,NaN
std,2.000001e+00,1.935761e+08,NaN,NaN,NaN,NaN
min,0.000000e+00,1.467810e+09,NaN,NaN,NaN,NaN
25%,0.000000e+00,1.956916e+09,NaN,NaN,NaN,NaN
50%,2.000000e+00,2.002102e+09,NaN,NaN,NaN,NaN
75%,4.000000e+00,2.177059e+09,NaN,NaN,NaN,NaN


In [ ]:
df.info()
df.shape
df.index

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


RangeIndex(start=0, stop=1600000, step=1)

In [ ]:
df['flag'].value_counts()


NO_QUERY    1600000
Name: flag, dtype: int64

In [ ]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
df['target']=['Negative' if x==0 else 'Positive' for x in df['target']]

In [ ]:
df["target"].value_counts()

Negative    800000
Positive    800000
Name: target, dtype: int64

In [ ]:
#Vamos a Preprocesar la Información
stop_words=stopwords.words('english')
port_stem=PorterStemmer()
def pre_process(content):
  process_content=re.sub(r'[^a-zA-Z\s]',"",content)
  process_content=process_content.lower()
  process_content=process_content.split(' ')
  process_content=[port_stem.stem(word) for word in process_content if word not in stop_words]
  process_content= " ".join(process_content)
  return process_content
df['processed_stemed_data']=df['text'].apply(pre_process)


In [ ]:
df.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text',
       'processed_stemed_data'],
      dtype='object')

In [ ]:
x=df['processed_stemed_data']
y=df['target']
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,stratify=y)
vectorizer=TfidfVectorizer()
X_train_vect=vectorizer.fit_transform(X_train)
X_test_vect=vectorizer.transform(X_test)

In [ ]:
model=LogisticRegression(max_iter=1600000)
model.fit(X_train_vect,Y_train)

LogisticRegression(max_iter=1600000)

In [ ]:
model.score(X_test_vect,Y_test)

0.783796875

In [ ]:
pred=model.predict(X_train_vect)
print(accuracy_score(Y_train,pred))

0.81722265625


In [ ]:

#https://www.kaggle.com/code/vasudevsrivastava/sentiment-analysis-beginner-deep-learning
#Vamos a guardar el modelo par autilizarlo más tarde

joblib.dump(model,"dump_sentiment_analysis_logistic_model.joblib")
joblib.dump(vectorizer,"vectorizer_vocabulary.joblib")
sentiment_analysis_logistic_model=joblib.load('dump_sentiment_analysis_logistic_model.joblib')
vectorize_vocabulary=joblib.load("vectorizer_vocabulary.joblib")

In [ ]:
test_senti="i am bad"
test_senti=vectorize_vocabulary.transform([test_senti])
sentiment_analysis_logistic_model.predict(test_senti)[0]

'Negative'